In [1]:
import pandas as pd
# read data
df = pd.read_csv("logins.txt", sep="\t", parse_dates=[[0,1]], header=None, dtype_backend="pyarrow")
# rename cols
df.columns = ["datetime", "uname", "login"]
# split names
df[["First Letter", "Name"]] = df["uname"].str.split(".", expand=True)
# # convert to
# # df = df.convert_dtypes(dtype_backend="pyarrow")
# jdf = df.loc[df["First Letter"]=="j"]
# # get just the times in string format
# jdf["time"] = jdf["datetime"].dt.time
# # count number of unique login times by name
# jdf.groupby("Name")["time"].nunique().sort_values().tail(1)

In [2]:
df = df.set_index("datetime")
df = df["2021-06-01":"2021-08-30"]

In [3]:
df

,uname,login,First Letter,Name
datetime,,,,
2021-06-01 00:00:00,d.gamble,OUT,d,gamble
2021-06-01 00:00:00,d.conti,OUT,d,conti
2021-06-01 00:00:00,j.bono,OUT,j,bono
2021-06-01 00:00:00,m.pryor,OUT,m,pryor
2021-06-01 00:00:00,c.harnden,OUT,c,harnden
...,...,...,...,...
2021-08-30 23:59:00,m.ruiz,OUT,m,ruiz
2021-08-30 23:59:00,j.rudd,OUT,j,rudd
2021-08-30 23:59:00,v.campbell,OUT,v,campbell


In [4]:
"""There's one account that changed behavior in July in a
way that is hard to detect. The employee is cheating the
timecard system."""


# df = df.rename(columns={'0_1':'datetime', 2:'uname', 3:'login'})
# df = df.sort_values(by=['uname', 'datetime'])
jdf = df.reset_index()

# Initialize a column of the time since the user's last login

jdf['Last_login'] = jdf['datetime'].diff()

logins = jdf[jdf['login'] == 'IN']
logouts = jdf[jdf['login'] == 'OUT']


# Make sure first logout happens after first login
first_logout_after_login = logouts.groupby('uname').apply(
    lambda x: x[x['datetime'] > logins.loc[logins['uname'] == x.name].iloc[0]['datetime']])

# Calculate time differences
time_diff = (first_logout_after_login.set_index('uname')['datetime'] - logins.set_index('uname')['datetime']).reset_index().rename(columns={0: 'time_diff'})



In [5]:
time_diff

,uname,datetime
0,a.1,0 days 07:52:00
1,a.1,-1 days +07:55:00
2,a.1,-5 days +07:52:00
3,a.1,-6 days +07:56:00
4,a.1,-7 days +07:57:00
...,...,...
37186436,z.yasso,4 days 08:01:00
37186437,z.yasso,3 days 07:58:00
37186438,z.yasso,2 days 08:01:00
37186439,z.yasso,1 days 07:59:00


In [7]:
first_logout_after_login.drop("uname", axis=1, inplace=True)
first_logout_after_login = first_logout_after_login.reset_index()

In [ ]:
# first_logout_after_login = first_logout_after_login.drop("uname", axis=1)
# time_diff = time_diff.set_index("uname")

In [8]:
# Align time difference with date of clock out
result = first_logout_after_login.merge(time_diff)

KeyboardInterrupt: 

In [ ]:
assert False
result['logout_time'] = result['datetime'] + result['time_diff']


In [ ]:
result

,uname,datetime
0,a.1,0 days 07:52:00
1,a.1,-1 days +07:55:00
2,a.1,-5 days +07:52:00
3,a.1,-6 days +07:56:00
4,a.1,-7 days +07:57:00
...,...,...
37186436,z.yasso,4 days 08:01:00
37186437,z.yasso,3 days 07:58:00
37186438,z.yasso,2 days 08:01:00
37186439,z.yasso,1 days 07:59:00


In [ ]:
result.groupby("uname")["datetime"].mean()

uname
a.1          -1 days +22:39:23.580155498
a.10              0 days 07:59:45.600000
a.11           0 days 07:59:30.731707317
a.12           0 days 07:59:30.379746835
a.13           0 days 07:59:34.615384615
                         ...            
z.norman       0 days 07:59:42.439024390
z.shackley     0 days 07:59:15.365853658
z.sommers      0 days 07:59:32.112676056
z.staggers   -1 days +22:58:57.758215963
z.yasso        0 days 07:59:45.925925925
Name: datetime, Length: 6196, dtype: timedelta64[ns]

In [ ]:
df.groupby("uname")["Last_login_S"].var().sort_values()

uname
c.21            2.189097e+08
v.sullivan      2.199053e+08
c.macdougall    2.730384e+08
n.schaller      2.793684e+08
p.story         3.852455e+08
                    ...     
d.hernandez     1.069342e+10
h.meyer         1.084304e+10
m.ponds         1.369947e+10
j.bruner        1.716605e+10
d.thorn                  NaN
Name: Last_login_S, Length: 6279, dtype: float64

In [ ]:
# how do you cheat the timecard system?
# check length of login times

login_lengths = df_ju2a[""]

In [ ]:
means = jdf.groupby("Name")["datetime"].mean()


In [ ]:
len(df[df["First Letter"]=="j"]["Name"].unique())

853

Index(['datetime', 'uname', 'login', 'First Letter', 'Name'], dtype='object')

In [ ]:

# means

Name
1          2021-07-04 19:20:53.419354880
10         2021-06-30 21:13:11.156250112
100        2021-06-29 10:33:15.993690880
101        2021-06-30 16:34:45.745341440
102        2021-06-29 22:48:51.774193664
                        ...             
yokoyama   2021-07-02 03:00:58.000000000
young      2021-06-29 13:20:45.864406784
yucha      2021-06-29 20:37:20.718749952
zapata     2021-07-03 00:20:56.369426688
zaring     2021-07-04 12:49:52.500000000
Name: datetime, Length: 853, dtype: datetime64[ns]

In [ ]:
jdf["time"] = jdf["datetime"].dt.time
jdf.groupby("Name")["time"].nunique().sort_values().max()

C:\Users\Morri\AppData\Local\Temp\ipykernel_21764\3559795783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jdf["time"] = jdf["datetime"].dt.time


Name
lyle            26
parker          26
150             27
woodruff        27
decook          27
              ... 
dobrinin        76
23              77
hughes          77
salano          95
steinberger    205
Name: time, Length: 853, dtype: int64

In [ ]:
names = jdf["Name"].unique()
varis = {}
for name in names:
    varis[name] = (jdf["datetime"] - means[name]).dt.total_seconds()**2

pd.Series(varis)


KeyboardInterrupt: 

In [ ]:
# jdf["Time"] = jdf["0_1"].dt.time.copy(deep=True)
jdfin = jdf.loc[jdf["login"]=="IN"] #.copy(deep=True)
jdfin.groupby("Name")["time"].mean()
# .groupby("Name")["0_1"].dt.total_seconds().mean()
jdfin

TypeError: unsupported operand type(s) for +: 'datetime.time' and 'datetime.time'